In [5]:
import os
import sys
import subprocess
import glob
import json
import pandas
import pickle
import pprint
from axe import axe # This package needs HELLA work.
from pathlib import Path
from collections import defaultdict # I like this collection. I am going to use this much more often.
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Blast import NCBIXML
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastpCommandline

In [6]:
RUN_BLAST = True
PARSE_BLAST = True

In [7]:
def parse_blast_results(blast_results : str, mode) -> dict:
    file_size = os.path.getsize(blast_results)
    if file_size == 0:
        return None
    results = {}
    with open(blast_results, "r") as f:
        blast_records = NCBIXML.parse(f)
        for record in blast_records:
            results[record.query] = []
            for alignment in record.alignments:
                #identity_and_coverage = calculate_percent_identity_and_coverage(alignment)
                #results[record.query].append((alignment.hit_id, identity_and_coverage))
                for hsp in alignment.hsps:
                    results[record.query].append({'contig_id' : record.query,
                                                  'contig_len' : record.query_length,
                                                  'hit_id' : alignment.hit_id,
                                                  'hit_title' : alignment.title,
                                                  'hit_def' : alignment.hit_def,
                                                  'hit_expect' : hsp.expect,
                                                  'hit_score' : hsp.score,
                                                  'hit_query_start' : hsp.query_start,
                                                  'hit_query_end' : hsp.query_end,
                                                  'hit_subject_start' : hsp.sbjct_start,
                                                  'hit_subject_end' : hsp.sbjct_end,
                                                  'hit_idents' : hsp.identities,
                                                  'hit_pos' : hsp.positives,
                                                  'hit_gaps' : hsp.gaps,
                                                  'hit_frame' : hsp.frame,
                                                  'hit_query' : hsp.query,
                                                  'hit_subject' : hsp.sbjct,
                                                  })
    return results

In [8]:
testing_set_file = 'dbs/Minimal-PF32-testor-set-March2021.faa'


In [26]:
# THIS NEEDS TO BE DONE BETTER, I DO NOT KNOW HOW TO DO THAT AT THIS POINT BUT I WILL FIGURE IT OUT.
# MAYBE CALLING FROM A LIST OF PATHS OR SOMETHING?
# TBD {{TODO}}

#### Define Inputs and Paths ####
# get CURRENT working directory
init_cwd = os.getcwd()
# Set working directory
working_dir = '/home/mf019/borrelia_plasmid_classifier_v3'
# path out the databases
databases_dir = f'{working_dir}/dbs'
whole_plasmid_db = f'{databases_dir}/plasmid_db_nf/plasmid_db'
pf32_db = f'{databases_dir}/pf32_db/pfam32db'
# lets specify where parsing tables are to be found!
parsing_tables_dir = f'{working_dir}/parsing_tables'
# set assemblies directory # DO I WANT TO ARGV THIS?
assemblies_dir = f'{working_dir}/assemblies' #f'{working_dir}/assemblies'
shortread_dir  = f'{assemblies_dir}/shortread'
longread_dir = f'{assemblies_dir}/longread'
shortread_contigs  = f'{shortread_dir}/contigs'
longread_contigs = f'{longread_dir}/contigs'
shortread_annotations  = f'{shortread_dir}/annotation'
longread_annotations = f'{longread_dir}/annotation'
output_dir = f'{working_dir}/output'
blast_results_dir = f'{output_dir}/blast_results_v4'
pf32_blast_results = f'{blast_results_dir}/pf32'
plasmid_blast_results = f'{blast_results_dir}/whole_plasmid'

In [24]:
print('analysis inputs')
print(f'CWD: {init_cwd}')
print(f'Working directory: {working_dir}')
print(f'Databases directory: {databases_dir}')
print(f'Plasmid database: {whole_plasmid_db}')
print(f'PF32 database: {pf32_db}')
print(f'Parsing tables directory: {parsing_tables_dir}')
print(f'Assemblies directory: {assemblies_dir}')
print(f'Shortread directory: {shortread_dir}')
print(f'Longread directory: {longread_dir}')
print(f'Shortread contigs directory: {shortread_contigs}')
print(f'Shortread annotations directory: {shortread_annotations}')
print(f'Longread contigs directory: {longread_contigs}')
print(f'Longread annotations directory: {longread_annotations}')
print(f'Output directory: {output_dir}')
print(f'Whole contig blast results directory: {blast_results_dir}')
print(f'PF32 Blast results: {pf32_blast_results}')
print('analysis begin')


analysis inputs
CWD: /home/mf019/borrelia_plasmid_classifier_v3
Working directory: /home/mf019/borrelia_plasmid_classifier_v3
Databases directory: /home/mf019/borrelia_plasmid_classifier_v3/dbs
Plasmid database: /home/mf019/borrelia_plasmid_classifier_v3/dbs/plasmid_db_nf/plasmid_db
PF32 database: /home/mf019/borrelia_plasmid_classifier_v3/dbs/pf32_db/pf32db
Parsing tables directory: /home/mf019/borrelia_plasmid_classifier_v3/parsing_tables
Assemblies directory: /home/mf019/borrelia_plasmid_classifier_v3/assemblies
Shortread directory: /home/mf019/borrelia_plasmid_classifier_v3/assemblies/shortread
Longread directory: /home/mf019/borrelia_plasmid_classifier_v3/assemblies/longread
Shortread contigs directory: /home/mf019/borrelia_plasmid_classifier_v3/assemblies/shortread/contigs
Shortread annotations directory: /home/mf019/borrelia_plasmid_classifier_v3/assemblies/shortread/annotation
Longread contigs directory: /home/mf019/borrelia_plasmid_classifier_v3/assemblies/longread/contigs
Lon

In [17]:
os.listdir(assemblies_dir)

['UCT30.fasta',
 'URI41.fasta',
 'URI44.fasta',
 'UWI247P.fasta',
 'URI111.fasta',
 '.ipynb_checkpoints',
 'UWI247.fasta',
 'URI91.fasta',
 'URI48H.fasta',
 'UWI248P.fasta',
 'UCT29H.fasta',
 'UCT92.fasta',
 'UCT50H.fasta',
 'UNY149P.fasta',
 'URI47H.fasta',
 'URI40.fasta',
 'URI107H.fasta',
 'UNY193P.fasta',
 'UNY149.fasta',
 'UCT109.fasta',
 'UCT29.fasta',
 'URI120.fasta',
 'URI33H.fasta',
 'UNY203P.fasta',
 'URI91H.fasta',
 'URI88H.fasta',
 'URI44H.fasta',
 'URI89.fasta',
 'UCT109H.fasta',
 'UNY172P.fasta',
 'URI101.fasta',
 'URI33.fasta',
 'UCT110H.fasta',
 'UCT31.fasta',
 'UWI248.fasta',
 'URI101H.fasta',
 'URI86H.fasta',
 'URI42.fasta',
 'URI118H.fasta',
 'ESI26H.fasta',
 'URI34.fasta',
 'URI47.fasta',
 'UNY203.fasta',
 'UCT35H.fasta',
 'URI103H.fasta',
 'URI103.fasta',
 'UCT30H.fasta',
 'UNY208.fasta',
 'UCT113H.fasta',
 'UCT96.fasta',
 'UNY172.fasta',
 'ESI26.fasta',
 'UCT31H.fasta',
 'URI88.fasta',
 'UNY208P.fasta',
 'URI87H.fasta',
 'URI87.fasta',
 'URI42H.fasta',
 'URI112.fa

In [18]:
# get annotations for each method
sr_gbs = glob.glob(f'{shortread_annotations}/*/*.gbff')
print(f'Found {len(sr_gbs)} shortread gbff files')
lr_gbs = glob.glob(f'{longread_annotations}/*/*.gbff')
print(f'Found {len(lr_gbs)} longread gbff files')

Found 49 shortread gbff files
Found 49 longread gbff files


In [20]:
assembly_dict = {}
for assembly in sr_gbs:
    isolate_name = str(assembly.split('/')[-1].strip('.gbff'))
    print(isolate_name)
    assembly_dict[isolate_name] = {'shortread' : defaultdict(dict), 'longread' : defaultdict(dict), 'longread_name' : None}
    for contig in SeqIO.parse(assembly, 'genbank'):
        #print(contig.name)
        assembly_dict[isolate_name]['shortread'][contig.name] = defaultdict(dict)
        assembly_dict[isolate_name]['shortread'][contig.name]['seqrecord'] = contig
        assembly_dict[isolate_name]['shortread'][contig.name]['wp_hits'] = []
        #print(f'added keys for wphits to {isolate_name} {contig.name}')
        assembly_dict[isolate_name]['shortread'][contig.name]['pf32_hits'] = []
        #print(f'added keys for pf32hits to {isolate_name} {contig.name}')
for assembly in lr_gbs:
    isolate_name = assembly.split('/')[-1].strip('.gbff')
    print(isolate_name)
    sr_name = isolate_name[:-1:]
    assembly_dict[sr_name]['longread_name'] = isolate_name
    for contig in SeqIO.parse(assembly, 'genbank'):
        #print(contig.name)
        assembly_dict[sr_name]['longread'][contig.name] = defaultdict(dict)
        assembly_dict[sr_name]['longread'][contig.name]['seqrecord'] = contig
        assembly_dict[sr_name]['longread'][contig.name]['wp_hits'] = []
        #print(f'added keys for wphits to {isolate_name} {contig.name}')
        assembly_dict[sr_name]['longread'][contig.name]['pf32_hits'] = []
        #print(f'added keys for pf32hits to {isolate_name} {contig.name}')
print('dictionary construction complete! pickling!')
pickle.dump(assembly_dict, open(f'{output_dir}/assembly_dict_v4.pkl', 'wb'))
print(f'dictionary pickled to: {output_dir}/assembly_dict_v4.pkl')

URI47
URI103
UCT31
UCT92
URI46
UWI247
UCT30
URI44
URI87
UCT50
UWI283
URI107
UCT110
URI48
UNY203
UWI263
URI117
UCT35
URI111
URI33
URI86
UNY149
UCT96
URI56
URI118
UNY172
URI112
UNY208
URI89
URI102
URI36
UWI248
ESI26
URI40
URI39
UNY169
UCT113
URI101
URI42
UCT109
UCT32
URI41
URI88
URI91
UCT29
URI34
URI120
UNY193
URI93
URI87H
URI34H
URI88H
URI33H
UCT110H
URI39H
URI91H
UCT35H
UWI247P
URI120H
URI107H
UWI263P
URI89H
URI42H
URI44H
UCT109H
URI40H
URI117H
URI47H
URI86H
URI36H
UNY208P
ESI26H
UCT31H
URI56H
UCT30H
URI103H
UCT29H
URI112H
UWI248P
UNY203P
UCT96H
UCT32H
UNY193P
UCT113H
URI93H
UNY169P
UWI283P
URI102H
URI41H
UNY172P
UNY149P
UCT92H
URI118H
UCT50H
URI101H
URI46H
URI48H
URI111H
dictionary construction complete! pickling!
dictionary pickled to: /home/mf019/borrelia_plasmid_classifier_v3/output/assembly_dict_v4.pkl


In [30]:
if RUN_BLAST is True:
    print(str('Start Blastin'))
    for isolate in assembly_dict:
        input_files = [] # empty list to store input files
        lr_data = assembly_dict[isolate]['longread']
        sr_data = assembly_dict[isolate]['shortread']
        lr_id = assembly_dict[isolate]['longread_name']
        #lr_id = isolate # since we're operating on lr first
        input_files.append(f'{longread_annotations}/{lr_id}/{lr_id}.fna') # add lr contigs to list
        if sr_data == 'NOT AVAILABLE':
            print(f'No shortread annotations found for {isolate}')
            sr_id = None
        else:
            sr_id = isolate#[:-1:]
            input_files.append(f'{shortread_annotations}/{sr_id}/{sr_id}.fna')
        print(f'setting up blast commands for {isolate}')
        for file in input_files:
            name = Path(file).stem
            wp_output_file = f'{plasmid_blast_results}/{name}_whole_plasmid.xml'
            wp_blast_cmd = f'blastn -query {file} -task "blastn" '
            wp_blast_cmd += f'-db {whole_plasmid_db} '
            wp_blast_cmd += f' -out {wp_output_file} '
            wp_blast_cmd += "-evalue 1e-100 -num_threads 60 -outfmt 5 -max_target_seqs 5 -max_hsps 10"
            print(f'Running blast command:\n{wp_blast_cmd}')
            subprocess.run(wp_blast_cmd, shell=True) # run it
            print(f'whole plasmid blast results written to: {wp_output_file}')
            print(f'setting up blast command for pf32 blast')
            pf_output_file = f'{pf32_blast_results}/{name}_pf32.xml'
            pf_blast_cmd = f'blastx -query {file} -task "blastx" '
            pf_blast_cmd += f'-db {pf32_db} '
            pf_blast_cmd += f' -out {pf_output_file} '
            pf_blast_cmd += "-evalue 1e-100 -num_threads 60 -outfmt 5 -max_target_seqs 5 -max_hsps 10"
            subprocess.run(pf_blast_cmd, shell=True, check=False) # run it, but don't check for errors to avoid stopping the script
            print(f'pf32 blast results written to: {pf_output_file}')
    print('saddle up, we are done here!')
else:
    print('Blast already run, skipping blast')

Start Blastin
setting up blast commands for URI47
Running blast command:
blastn -query /home/mf019/borrelia_plasmid_classifier_v3/assemblies/longread/annotation/URI47H/URI47H.fna -task "blastn" -db /home/mf019/borrelia_plasmid_classifier_v3/dbs/plasmid_db_nf/plasmid_db  -out /home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/whole_plasmid/URI47H_whole_plasmid.xml -evalue 1e-100 -num_threads 60 -outfmt 5 -max_target_seqs 5 -max_hsps 10
whole plasmid blast results written to: /home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/whole_plasmid/URI47H_whole_plasmid.xml
setting up blast command for pf32 blast
pf32 blast results written to: /home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/pf32/URI47H_pf32.xml
Running blast command:
blastn -query /home/mf019/borrelia_plasmid_classifier_v3/assemblies/shortread/annotation/URI47/URI47.fna -task "blastn" -db /home/mf019/borrelia_plasmid_classifier_v3/dbs/plasmid_db_nf/plasmid_db  -out /home/mf019/bor

## Let's parse our blast hits now!

In [31]:
pf32_results = glob.glob(f'{pf32_blast_results}/*.xml')
wp_results = glob.glob(f'{plasmid_blast_results}/*.xml')
print(f'Found {len(pf32_results)} pf32 blast results')
print(f'Found {len(wp_results)} whole plasmid blast results')

Found 98 pf32 blast results
Found 98 whole plasmid blast results


In [32]:
if PARSE_BLAST is True:
    for result in pf32_results:
        print(result)
        name = Path(result).stem.split("_")[0]
        if name.endswith(('H','P')):
            print(f'input is longread: {name}')
            name = name[:-1:] # chop the char for dict lookup.
            current_asm = assembly_dict[name]['longread']
        else:
            print(f'input is shortread: {name}')
            current_asm = assembly_dict[name]['shortread']
        #print(current_asm.keys())
        blasthits = parse_blast_results(result, "pf")
        for hit in blasthits:
            if blasthits[hit] is None:
                print(f'No hits for {hit}')
            else:
                for h in blasthits[hit]:
                    #print(h)
                    #print(h['contig_id'])
                    #print(current_asm[h['contig_id']])
                    current_asm[h['contig_id']]['pf32_hits'].append(h)

    for result in wp_results:
        print(result)
        name = Path(result).stem.split("_")[0]
        if name.endswith(('H','P')):
            print(f'input is longread: {name}')
            name = name[:-1:] # chop the char for dict lookup.
            current_asm = assembly_dict[name]['longread']
        else:
            print(f'input is shortread: {name}')
            current_asm = assembly_dict[name]['shortread']
        #print(current_asm.keys())
        blasthits = parse_blast_results(result, "wp")
        for hit in blasthits:
            if blasthits[hit] is None:
                print(f'No hits for {hit}')
            else:
                for h in blasthits[hit]:
                    #print(h)
                    #print(h['contig_id'])
                    #print(current_asm[h['contig_id']])
                    current_asm[h['contig_id']]['wp_hits'].append(h)

/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/pf32/UCT35H_pf32.xml
input is longread: UCT35H
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/pf32/ESI26_pf32.xml
input is shortread: ESI26
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/pf32/ESI26H_pf32.xml
input is longread: ESI26H
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/pf32/UCT32H_pf32.xml
input is longread: UCT32H
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/pf32/UNY169P_pf32.xml
input is longread: UNY169P
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/pf32/UWI248P_pf32.xml
input is longread: UWI248P
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/pf32/URI34H_pf32.xml
input is longread: URI34H
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/pf32/URI111_pf32.xml
input is shortread: URI111
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/pf32/UCT29H_pf32.

In [35]:
#pickle.dump(assembly_dict, open(f'{output_dir}/assembly_dict_v4_parsed.pkl', 'wb'))
assembly_dict = pickle.load(open(f'{output_dir}/assembly_dict_v4_parsed.pkl', 'rb'))

In [36]:
# Ok let's crack open this jar of pickled plasmids.
parsing_table = pickle.load(open(f'{parsing_tables_dir}/blast_parsing_dict.pkl', 'rb'))
# ok now let's set up the columns for our matrix and drop the synthetic vector I added a while back...
plasmids_in_db = {k : v for k, v in parsing_table.items()}
matrix_cols = []
for k in plasmids_in_db:
    matrix_cols.append(plasmids_in_db[k]['name'])
matrix_cols = list(set(matrix_cols))
matrix_cols.sort()
matrix_cols.remove('pBSV2')
matrix_cols.remove('lp21-cp9')
print(matrix_cols)

['chromosome', 'cp26', 'cp32-1', 'cp32-1+5', 'cp32-10', 'cp32-11', 'cp32-12', 'cp32-2', 'cp32-3', 'cp32-3+10', 'cp32-4', 'cp32-5', 'cp32-5+1', 'cp32-5-1', 'cp32-6', 'cp32-7', 'cp32-8', 'cp32-9', 'cp32-9-4', 'cp9', 'cp9-3', 'lp17', 'lp21', 'lp25', 'lp28-1', 'lp28-11', 'lp28-2', 'lp28-3', 'lp28-4', 'lp28-5', 'lp28-6', 'lp28-7', 'lp28-8', 'lp28-9', 'lp36', 'lp38', 'lp5', 'lp54', 'lp56']


In [37]:
for key in parsing_table:
    if 'cp32' in parsing_table[key]['name']:
        #print(parsing_table[key])
        pass

In [112]:
def get_hit_id(hit_title):
    """Extract hit id from hit title"""
    #print(hit_title)
    hit_id = hit_title.split(' ')[0].split("_")[-1]
    #print(f'{hit_title} => {hit_id}')
    if "|" in hit_id:
        if 'pdb' in hit_id: # I do not know why this singular db entry is different. thank you very much blast :)
            hit_id = hit_id.split("|")[-1]
        else:
            hit_id = hit_id.split("|")[1]
    return hit_id

# This needs defaultdict() from collections.
# I literally already have this above....... Why did I rewrite this? # nvm i remember. it needed it, dropped the redundancy above.
def parse_hit(hit, **kwargs):
    """output the relevant information from a hit
    ['hit_id', 'contig_id', 'contig_len', 'hit_title', 'hit_def', 'bitscore', 'e_score',
    'query_start', 'query_end', 'ref_start', 'ref_end',
    'hit_pos', 'idents', 'gaps', 'frame', 'query_seq']
    enter a list of keys to return in the return_list kwarg to return only those keys
    ex:
    parse_hit(hit, return_list=['hit_id', 'contig_id', 'contig_len', 'hit_title'])
    """
    output_list = []
    if 'return_list' in kwargs.keys():
        return_list = kwargs['return_list']
    #print("inside parse_hit")
    #print(f"Hit ID: {hit['hit_id']}")
    hit_id = get_hit_id(hit['hit_id'])
    # I feel like I could make a simple class for this so that I could access the values in dot notation. i.e. hit.hit_id, hit.contig_id, etc.
    hit_dict = {
        'hit_id'  : hit_id,
        'contig_id' : hit['contig_id'],
        'contig_len' : hit['contig_len'],
        'hit_title' : hit['hit_title'],
        'hit_def' : hit['hit_def'],
        'bitscore' : hit['hit_score'],
        'e_score' : hit['hit_expect'],
        'query_start' : hit['hit_query_start'],
        'query_end' : hit['hit_query_end'],
        'ref_start' : hit['hit_subject_start'],
        'ref_end' : hit['hit_subject_end'],
        'hit_pos': hit['hit_pos'],
        'idents' : hit['hit_idents'],
        'gaps' : hit['hit_gaps'],
        'frame' : hit['hit_frame'],
        'query_seq' : hit['hit_query']}
    for key in return_list:
        output_list.append(hit_dict[key])
    return output_list

def get_matching_feature(contig_start, contig_end, features):
    """if it exists, find the annotation feature present at the pf32 hit coordinates"""
    for feature in features:
        if feature.type == 'CDS' or feature.type == 'gene':
            if feature.location.start - 10 <= contig_start <= feature.location.end + 10 and feature.location.start - 10 <= contig_end <= feature.location.end + 10:
                return feature
    return None

def process_contig(contig_obj, contig_id, method, isolate, lr_name, parsing_dict, assembly_dict,cutoff):
    """Process a single contig"""
    # {{TODO}}
    # The logic is okay-ish but needs MAJOR revision and optimization for readability.
    # I should honestly strip this down to the most atomic form to work on ONE SINGULAR INPUT FILE. THAT IS MY ISSUE. NOT THE NUMBER OF FILES BUT THE FACT THAT MY SOLUTIONS AND TOOLS
    # ARE BUILT TO HANDLE MULTIPLE FILES AT ONCE AND THESE FILES ARE RARELY CONSISTENT. I can just work on one file at a time and then iterate over them in a loop.
    # This is so obvious but I don't know why I haven't realized that yet.
    # This is also basic C.S. and idk how I missed this. I'm writing complex, fragile, rigid code when I could be writing simple, robust, flexible code. This is all from the fact that
    # I am trying to do too much at once. I need to slow down and take it one step at a time. I need to break it down into the smallest possible components and work on those and prioritize
    # everything working together for a single input.
    # THEN: I can include exceptions and catch faulty formatting.
    # This should make things so much easier to manage and debug.
    # Gosh, who even let me write code like this? I've been kneecapping myself at every step

    sr_plasmids = []
    lr_plasmids = []
    pf_lines = ['']
    wp_lines = ['']
    use_wp = False
    contig_len = len(assembly_dict[isolate][method][contig_id]['seqrecord'].seq)
    if contig_len <= cutoff:
        #print(f'{contig_id} is too short, len < 2500')
        pf_lines.append(f'{contig_id}\tlt_2500bp\n')
        wp_lines.append(f'{contig_id}\tlt_2500bp\n')
        return sr_plasmids, lr_plasmids, pf_lines, wp_lines
    else:
        if contig_len > 100000:
            #print(f'{contig_id} is prob chromosome or chromosome fragment')
            use_wp = True
        else:
            if len(contig_obj.get("pf32_hits")) > 0:
                pf32_hits = contig_obj.get("pf32_hits")
                for pf32hit in pf32_hits:
                    if method == 'shortread': # just tack on the first hit.
                        #print(isolate)
                        sr_plasmids.append(get_hit_id(pf32_hits[0]['hit_id']))
                    else:
                        #print(isolate)
                        lr_plasmids.append(get_hit_id(pf32_hits[0]['hit_id']))
                    # ok now let's get the stuff from our results
                    parsed_pfhit_id, hit_contig_id, bitscore, escore,  contig_start, contig_end, ref_start, ref_end = parse_hit(pf32hit, return_list=['hit_id', 'contig_id', 'bitscore', 'e_score', 'query_start', 'query_end', 'ref_start', 'ref_end'])
                    matching_feature = get_matching_feature(contig_start, contig_end, assembly_dict[isolate][method][contig_id]['seqrecord'].features)
                    if matching_feature is not None:# I'm gonna have a stroke this literaelly donw't make sense. This also can probably either fit into the matching feature function
                        if 'gene' in matching_feature.qualifiers: # or possibly be it's own. I'm leaning towards the former
                            annotated_feat = matching_feature.qualifiers['gene']
                        elif 'locus_tag' in matching_feature.qualifiers:
                            annotated_feat = matching_feature.qualifiers['locus_tag']
                        elif 'product' in matching_feature.qualifiers:
                            annotated_feat = matching_feature.qualifiers['product']
                        else:
                            annotated_feat = 'not gene locus or cds'
                        #print(f'{contig_id} {matching_feature}')
                        annofeat_start = matching_feature.location.start
                        annofeat_end = matching_feature.location.end
                    else:
                        annotated_feat = 'no annotated feature'
                        annofeat_start = 'NA'
                        annofeat_end = 'NA'
                    pf_line = f'{contig_id}\t{parsed_pfhit_id}\t{contig_len}\t{bitscore}\t{escore}\t{contig_start}\t{contig_end}\t{annotated_feat}\t{annofeat_start}\t{annofeat_end}\n'
                    pf_lines.append(pf_line)
            else:
                pf_lines.append(f'{contig_id}\tNo pf32 hits!\n')
        if len(contig_obj.get("wp_hits")) > 0:
            wp_hits = contig_obj.get("wp_hits")
            for wp_hit in wp_hits:
                #print(isolate)
                #print(f'ID: {wp_hit["hit_id"]}, TITLE: {wp_hit["hit_title"]}, DEF: {wp_hit["hit_def"]}')
                wp_hit_id = get_hit_id(wp_hit['hit_id'])
                wp_hit_name = parsing_dict[wp_hit_id]['name']
                parsed_wphit_id, wphit_contig_id, bitscore, escore, query_start, query_end, ref_start, ref_end= parse_hit(wp_hit, return_list=['hit_id', 'contig_id', 'bitscore', 'e_score', 'query_start', 'query_end', 'ref_start', 'ref_end'])
                expected_length = parsing_dict[wp_hit_id]['length']
                wp_line = f'{contig_id}\t{parsed_wphit_id}\t{wp_hit_name}\t{contig_len}\t{expected_length}\t{bitscore}\t{escore}\t{query_start}\t{query_end}\t{ref_start}\t{ref_end}\n'
                wp_lines.append(wp_line)
            if use_wp is True or len(contig_obj.get("pf32_hits")) == 0:
                if method == 'shortread':
                    #print(isolate)
                    sr_plasmids.append(parsing_dict[get_hit_id(wp_hits[0]['hit_id'])]['name'])
                else:
                    #print(isolate)
                    lr_plasmids.append(parsing_dict[get_hit_id(wp_hits[0]['hit_id'])]['name'])
        else:
            wp_lines.append(f'{contig_id}\tNo whole plasmid hits!\n')
    return sr_plasmids, lr_plasmids, pf_lines, wp_lines

def process_isolate(isolate, assembly_dict, output_dir, method, parsing_dict,cutoff):
    """Process a single isolate"""
    pf_header = 'contig_id\tparsed_pfhit_id\tcontig_len\tbitscore\tescore\tquery_start\tquery_end\tannotated_gene\tgene_start\tgenet_end\n'
    wp_header = 'contig_id\tparsed_wphit_id\twp_hit_name\tcontig_len\texpected_length\tbitscore\tescore\tquery_start\tquery_end\tref_start\tref_end\n'
    lr_name = assembly_dict[isolate]['longread_name']
    pfam_lines = []
    wpl_lines = []
    sr_hit_list = []
    lr_hit_list = []
    pfam_lines.insert(0, pf_header)
    wpl_lines.insert(0, wp_header)
    if method != 'longread_name':
        if method == 'shortread':
            iso_name = isolate
        else:
            iso_name = lr_name
        if os.path.exists(f'{output_dir}/plasmid_hits/') is False:
            os.mkdir(f'{output_dir}/plasmid_hits/')
        with open(f'{output_dir}/plasmid_hits/{iso_name}_plasmid_hits.tsv', 'w') as outfile:
            for contig in assembly_dict[isolate][method]:
                if contig != 'seqrecord':
                    contig_obj = assembly_dict[isolate][method][contig]
                    #print(f'processing {contig}')
                    sr_hits, lr_hits, pf_lines, wp_lines = process_contig(contig_obj, contig, method, isolate, lr_name, parsing_dict, assembly_dict,cutoff)
                    pfam_lines.extend(pf_lines)
                    wpl_lines.extend(wp_lines)
                    sr_hit_list.extend(sr_hits)
                    lr_hit_list.extend(lr_hits)
            outfile.writelines(pfam_lines)
            outfile.writelines(wpl_lines)
    return sr_hit_list, lr_hit_list

def process_assembly_dict(assembly_dict, output_dir, parsing_dict,cutoff):
    """Process the assembly dict"""
    sr_plasmids = defaultdict(dict)
    lr_plasmids = defaultdict(dict)
    for isolate in assembly_dict:
        sr_plasmids[isolate] = []
        for method in assembly_dict[isolate]:
            sr, lr = process_isolate(isolate, assembly_dict, output_dir, method, parsing_dict,cutoff)
            if method == 'longread':
                iso_name = assembly_dict[isolate]['longread_name']
                lr_plasmids[iso_name] = []
                lr_plasmids[iso_name].extend(lr)
            else:
                sr_plasmids[isolate].extend(sr)
            #print(f'added lr plasmids for {isolate}')
    return sr_plasmids, lr_plasmids


In [113]:
sr_plasmids, lr_plasmids = {}, {}
sr_plasmids, lr_plasmids = process_assembly_dict(assembly_dict, f"{output_dir}/blast_results_v4", parsing_table, 2500)

for k in lr_plasmids:
    print(f'longread: {k} : len:{len(list(set(lr_plasmids[k])))} : {list(set(lr_plasmids[k]))}')
    lr_plasmids[k] = list(set(lr_plasmids[k]))
    print(f'shortread: {k[:-1]} : len: {len(list(set(sr_plasmids[k[:-1]])))}{list(set(sr_plasmids[k[:-1]]))}')
    sr_plasmids[k[:-1]] = list(set(sr_plasmids[k[:-1]]))

dicts_of_results = [sr_plasmids, lr_plasmids]
plasmid_presence_matrix = []
for dict1 in dicts_of_results:
    for k in dict1:
        #print(k)
        for v in dict1[k]:
            #print(v)
            plasmid_presence_matrix.append((k,v))

plasmid_df = pandas.DataFrame(plasmid_presence_matrix, columns=['key','val']).set_index(['key'])
plasmid_count_matrix = plasmid_df.pivot_table(index='key', columns='val',aggfunc=len,fill_value=0,sort=True)
plasmid_count_matrix['Sum'] = plasmid_count_matrix.sum(axis=1)
print(plasmid_count_matrix)

plasmid_count_matrix.to_csv(f'{output_dir}/plasmid_presence_matrix_v4_2500cutoff.csv')

longread: URI47H : len:19 : ['cp32-12', 'cp32-4', 'lp28-6', 'cp9', 'cp26', 'cp32-7', 'cp32-6', 'cp32-3', 'lp54', 'lp28-1', 'cp32-11', 'lp28-3', 'lp17', 'lp28-4', 'cp32-9', 'cp32-5', 'chromosome', 'cp32-8', 'lp38']
shortread: URI47 : len: 23['lp25', 'cp32-12', 'lp21', 'lp28-6', 'cp32-2', 'cp9', 'cp26', 'cp32-7', 'cp32-6', 'cp32-3', 'lp54', 'lp28-1', 'cp32-11', 'lp28-5', 'lp28-3', 'lp17', 'lp28-4', 'cp32-9', 'cp32-5', 'chromosome', 'lp36', 'cp32-8', 'lp38']
longread: URI103H : len:18 : ['cp32-7', 'cp32-6', 'cp32-9', 'cp32-5', 'cp32-12', 'chromosome', 'lp25', 'lp28-1', 'cp32-3', 'cp32-4', 'lp38', 'lp28-6', 'cp32-11', 'lp17', 'lp28-3', 'lp28-4', 'lp54', 'cp26']
shortread: URI103 : len: 21['lp25', 'cp32-12', 'lp21', 'lp28-6', 'cp32-2', 'cp26', 'cp32-7', 'cp32-6', 'cp32-3', 'lp54', 'lp28-1', 'cp32-11', 'lp28-5', 'lp28-3', 'lp17', 'lp28-4', 'cp32-9', 'cp32-5', 'chromosome', 'lp36', 'lp38']
longread: UCT31H : len:21 : ['lp25', 'lp21', 'cp32-4', 'cp9', 'cp32-1', 'cp26', 'cp32-7', 'cp32-6', 'lp2

In [54]:
sr_plasmids, lr_plasmids = {}, {}
sr_plasmids, lr_plasmids = process_assembly_dict(assembly_dict, output_dir, parsing_table, 2500)

for k in lr_plasmids:
    print(f'longread: {k} : len:{len(list(set(lr_plasmids[k])))} : {list(set(lr_plasmids[k]))}')
    lr_plasmids[k] = list(set(lr_plasmids[k]))
    print(f'shortread: {k[:-1]} : len: {len(list(set(sr_plasmids[k[:-1]])))}{list(set(sr_plasmids[k[:-1]]))}')
    sr_plasmids[k[:-1]] = list(set(sr_plasmids[k[:-1]]))

dicts_of_results = [sr_plasmids, lr_plasmids]
plasmid_presence_matrix = []
for dict1 in dicts_of_results:
    for k in dict1:
        #print(k)
        for v in dict1[k]:
            #print(v)
            plasmid_presence_matrix.append((k,v))

plasmid_df = pandas.DataFrame(plasmid_presence_matrix, columns=['key','val']).set_index(['key'])
plasmid_count_matrix = plasmid_df.pivot_table(index='key', columns='val',aggfunc=len,fill_value=0,sort=True)
plasmid_count_matrix['Sum'] = plasmid_count_matrix.sum(axis=1)
print(plasmid_count_matrix)
#plasmid_count_matrix.to_csv(f'{output_dir}/plasmid_presence_matrix_v3_2000cutoff.csv')

longread: URI47H : len:19 : ['cp32-12', 'cp32-4', 'lp28-6', 'cp9', 'cp26', 'cp32-7', 'cp32-6', 'cp32-3', 'lp54', 'lp28-1', 'cp32-11', 'lp28-3', 'lp17', 'lp28-4', 'cp32-9', 'cp32-5', 'chromosome', 'cp32-8', 'lp38']
shortread: URI47 : len: 23['lp25', 'cp32-12', 'lp21', 'lp28-6', 'cp32-2', 'cp9', 'cp26', 'cp32-7', 'cp32-6', 'cp32-3', 'lp54', 'lp28-1', 'cp32-11', 'lp28-5', 'lp28-3', 'lp17', 'lp28-4', 'cp32-9', 'cp32-5', 'chromosome', 'lp36', 'cp32-8', 'lp38']
longread: URI103H : len:18 : ['cp32-7', 'cp32-6', 'cp32-9', 'cp32-5', 'cp32-12', 'chromosome', 'lp25', 'lp28-1', 'cp32-3', 'cp32-4', 'lp38', 'lp28-6', 'cp32-11', 'lp17', 'lp28-3', 'lp28-4', 'lp54', 'cp26']
shortread: URI103 : len: 21['lp25', 'cp32-12', 'lp21', 'lp28-6', 'cp32-2', 'cp26', 'cp32-7', 'cp32-6', 'cp32-3', 'lp54', 'lp28-1', 'cp32-11', 'lp28-5', 'lp28-3', 'lp17', 'lp28-4', 'cp32-9', 'cp32-5', 'chromosome', 'lp36', 'lp38']
longread: UCT31H : len:21 : ['lp25', 'lp21', 'cp32-4', 'cp9', 'cp32-1', 'cp26', 'cp32-7', 'cp32-6', 'lp2

In [63]:
plasmid_count_matrix[plasmid_count_matrix.get('cp32-5+1') ==1]



val,chromosome,cp26,cp32-1,cp32-1+5,cp32-10,cp32-11,cp32-12,cp32-13,cp32-2,cp32-3,...,lp28-6,lp28-7,lp28-8,lp28-9,lp36,lp38,lp5,lp54,lp56,Sum
key,,,,,,,,,,,,,,,,,,,,,
UCT110,1,1,0,0,0,1,0,0,1,1,...,1,0,0,0,1,1,0,1,0,22
URI34,1,1,0,0,1,0,1,1,0,0,...,1,1,0,0,1,1,1,1,0,22


In [ ]:
def throwexcept():
    raise Exception("this is an exception, stop hjere please")
throwexcept()

### With all due disrespect
what in the name of all things good and holy was I thinking when I wrote the below code????

In [67]:
#sr_plasmids = {}
#lr_plasmids = {}

sr_plasmids = defaultdict(dict)
lr_plasmids = defaultdict(dict)

for isolate in assembly_dict:
    lriso_name = assembly_dict[isolate]['longread_name']
    sr_plasmids[isolate] = []
    lr_plasmids[lriso_name] = []
    for method in assembly_dict[isolate]:
        if method == 'longread_name':
            break
        else:
            with open(f'{output_dir}/blast_results_v4/{isolate}_{method}_plasmid_hits_pf32.tsv', 'w') as f:
                with open(f'{output_dir}/blast_results_v4/{isolate}_{method}_plasmid_hits_wp.tsv', 'w') as g:
                    if method == 'shortread':
                        f.write(f'Assembly ID: {isolate}\n')
                        g.write(f'Assembly ID: {isolate}\n')
                        f.write('PF32 Hits:\n contig_id\thit_id\tcontig len\tbitscore\te_score\tcontig_start\tcontig_end\tgene_at_coords\tgene_start\tgene_end\n')
                        g.write('Whole Plasmid Hits:\ncontig_id\thit_name\thit_acc_id\tcontig_len\texpected_len\tbitscore\te_score\tref_start\tref_end\tcontig_start\tcontig_end\n')
                    else:
                        f.write(f'Assembly ID: {lriso_name}\n')
                        g.write(f'Assembly ID: {lriso_name}\n')
                        f.write('PF32 Hits:\n contig_id\thit_id\tcontig len\tbitscore\te_score\tcontig_start\tcontig_end\tgene_at_coords\tgene_start\tgene_end\n')
                        g.write('Whole Plasmid Hits:\ncontig_id\thit_name\thit_acc_id\tcontig_len\texpected_len\tbitscore\te_score\tref_start\tref_end\tcontig_start\tcontig_end\n')
                    for contig in assembly_dict[isolate][method]:
                        contig_len = len(assembly_dict[isolate][method][contig]['seqrecord'].seq)
                        if contig_len > 900000:
                                #print('prob chromosome')
                                f.write(f'{contig}\tchromosome\t{contig_len}\t"det_by_len"\t"det_by_len"\t"det_by_len"\t"det_by_len"\t"det_by_len"\t"det_by_len"\t"det_by_len"\n')
                        else:
                            #print(f'\ncurrent: {isolate}-{method}-{contig}')
                            if len(assembly_dict[isolate][method][contig]['seqrecord'].seq) < 2500:
                                #print('too short!')
                                f.write(f'{contig}\tcontig_len < 2500bp\n')
                            else:
                                if len(assembly_dict[isolate][method][contig].get("pf32_hits")) > 0:
                                    pfhit_title = assembly_dict[isolate][method][contig]["pf32_hits"]
                                    for pfhit in pfhit_title:
                                        #dict_keys(['contig_id', 'contig_len', 'hit_id', 'hit_title', 'hit_def', 'hit_expect', 'hit_score', 'hit_query_start', 'hit_query_end', 'hit_subject_start', 'hit_subject_end', 'hit_idents', 'hit_pos', 'hit_gaps', 'hit_frame', 'hit_query', 'hit_subject'])
                                        hit_id = pfhit["hit_title"].split(' ')[0].split("_")[-1]
                                        if "|" in hit_id:
                                            hit_id = hit_id.split("|")[-1]
                                        score = pfhit["hit_score"]
                                        contig_len = pfhit["contig_len"]
                                        contig_start = pfhit['hit_query_start']
                                        contig_end = pfhit['hit_query_end']
                                        e_score = hit['hit_expect']
                                        # Iterate over the features and find the one that matches the coordinates, this needs rewriting!!!!!
                                        matching_feature = None
                                        for feature in assembly_dict[isolate][method][contig]['seqrecord'].features:
                                            #print(feature)
                                            if feature.location.start - 10 <= contig_start <= feature.location.end + 10 and feature.location.start - 10 <= contig_end <= feature.location.end + 10:
                                                # Match found
                                                matching_feature = feature
                                                #print(matching_feature)
                                        # Print the annotation of the matching feature
                                        if matching_feature: # this should go within the above block.
                                            #['db_xref', 'product', 'locus_tag', 'protein_id', 'translation', 'codon_start', 'transl_table', 'inference', 'gene']
                                            if 'locus_tag' in matching_feature.qualifiers:
                                                feat_id = matching_feature.qualifiers['locus_tag'][0]
                                                product = matching_feature.qualifiers['product'][0]
                                                gene_start = matching_feature.location.start
                                                gene_end = matching_feature.location.end
                                                if 'gene' in matching_feature.qualifiers:
                                                    feat_id = matching_feature.qualifiers['gene'][0]
                                                    #print(f"found match => {gene_name}")
                                                else:
                                                    feat_id = product
                                                    #print(f"{isolate}{method} found match => {product}")
                                            else:
                                                print(f"{isolate}--{contig}__FEATURE'S DIFFERENT!: {matching_feature.qualifiers.keys()}") # this is a bit of a hack to see what's in the feature
                                                feat_id = 'NaN'
                                                gene_start = 'NaN'
                                                gene_end = 'NaN'
                                        else:
                                            pass
                                            #print('No feature found at the specified coordinates.')
                                    if "|" in pfhit_title[0]['hit_title'].split(' ')[0].split("_")[-1]:
                                        pf32_hit_id = pfhit_title[0]['hit_title'].split(' ')[0].split("_")[-1].split("|")[-1]
                                    else:
                                        pf32_hit_id = pfhit_title[0]['hit_title'].split(' ')[0].split("_")[-1]
                                    f.write(f'{contig}\t{pf32_hit_id}\t{contig_len}\t{score}\t{e_score}\t{contig_start}\t{contig_end}\t{feat_id}\t{gene_start}\t{gene_end}\n')
                                    if method == 'shortread':
                                        pftuple = (isolate, contig, "pf32", pf32_hit_id, contig_len,0)
                                        sr_plasmids[isolate].append(pftuple)
                                    else:
                                        pftuple = (lriso_name, contig, "pf32", pf32_hit_id, contig_len,0)
                                        lr_plasmids[lriso_name].append(pftuple)
                                else:
                                    f.write(f'{contig}\tNo PF32 hits\n')
                        if len(assembly_dict[isolate][method][contig].get("wp_hits")) > 0:
                            hit_title = assembly_dict[isolate][method][contig].get("wp_hits")
                            for hit in hit_title:
                                #dict_keys(['contig_id', 'contig_len', 'hit_id', 'hit_title', 'hit_def', 'hit_expect', 'hit_score', 'hit_query_start', 'hit_query_end', 'hit_subject_start', 'hit_subject_end', 'hit_idents', 'hit_pos', 'hit_gaps', 'hit_frame', 'hit_query', 'hit_subject'])
                                hit_id = hit["hit_title"].split('|')[1]
                                wp_hit = parsing_table[hit_id]['name']
                                score = hit["hit_score"]
                                contig_len = hit["contig_len"]
                                replicon_len = parsing_table[hit_id]['length']
                                ref_start = hit['hit_subject_start']
                                ref_end = hit['hit_subject_end']
                                e_score = hit['hit_expect']
                                contig_start = hit['hit_query_start']
                                contig_end = hit['hit_query_end']
                                #if len(assembly_dict[isolate][method][contig].get("pf32_hits")) == 0 or contig_len > 100000:
                                if len(assembly_dict[isolate][method][contig]['seqrecord'].seq) < 2500:
                                    g.write(f'{contig}\t contig_len < 2500bp\n')
                                else:           #contig_id\thit_name\thit_acc_id\tcontig_len\texpected_len\tbitscore\te_score\tref_start\tref_end\tcontig_start\tcontig_end\n
                                    g.write(f'{contig}\t{wp_hit}\t{hit_id}\t{contig_len}\t{replicon_len}\t{score}\t{e_score}\t{ref_start}\t{ref_end}\t{contig_start}\t{contig_end}\n')
                                    if method == 'shortread':
                                        name_tuple = (isolate, contig, "wp", parsing_table[hit_title[0]['hit_title'].split('|')[1]]['name'], contig_len, replicon_len)
                                        #print(name_tuple)
                                        sr_plasmids[isolate].append(name_tuple)
                                    else:
                                        name_tuple = (lriso_name, contig, "wp", parsing_table[hit_title[0]['hit_title'].split('|')[1]]['name'], contig_len, replicon_len)
                                        #print(name_tuple)
                                        lr_plasmids[lriso_name].append(name_tuple)
                        else:
                            pass
                            #print(f'{contig}\tNo whole plasmid hits')


URI103--NODE_17_length_7333_cov_478.155733__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
URI46--contig000014__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
UCT30--contig000016__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
UCT110--contig000010__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
UCT110--contig000016__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
UNY203--NODE_60_length_6810_cov_5.51115__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
URI117--contig000019__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
URI117--contig000022__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
UCT35--contig000015__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
UCT35--contig000015__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
UCT35--contig000015__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
URI33--contig000016__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
URI86--contig000013__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
URI118--contig000014__FEATURE'S DIFFERENT!: dict_keys(['mol_type'])
UNY172--B356_c

In [148]:
# Function to calculate overall percent identity and coverage using HSPs
def calculate_percent_identity_and_coverage(alignment):
    total_identities = 0
    total_alignment_length = 0
    covered_positions = set()

    for hsp in alignment.hsps:
        total_identities += hsp.identities
        total_alignment_length += hsp.align_length

        # Track covered positions in the reference
        covered_positions.update(range(hsp.sbjct_start, hsp.sbjct_end))

    # Calculate overall percent identity
    overall_percent_identity = (total_identities / total_alignment_length) * 100 if total_alignment_length != 0 else 0

    # Calculate coverage percentage
    reference_length = alignment.length
    coverage_percentage = (len(covered_positions) / reference_length) * 100 if reference_length != 0 else 0

    return overall_percent_identity, coverage_percentage, covered_positions

In [204]:
# Function to calculate overall percent identity and coverage using HSPs
def calculate_percent_identity_and_coverage(alignment):
    total_identities = 0
    total_alignment_length = 0
    covered_intervals = []

    for hsp in alignment.hsps:
        total_identities += hsp.identities
        total_alignment_length += hsp.align_length

        # Add HSP interval to the covered intervals list, make sure the coordinates are correct!
        start = min(hsp.sbjct_start, hsp.sbjct_end)
        end = max(hsp.sbjct_start, hsp.sbjct_end)

        # Add HSP interval to the covered intervals list
        covered_intervals.append((start, end))

    # Merge overlapping intervals to calculate the covered length
    merged_intervals = merge_intervals(covered_intervals)
    covered_length = sum(end - start for start, end in merged_intervals)

    # Calculate overall percent identity
    overall_percent_identity = (total_identities / total_alignment_length) * 100 if total_alignment_length != 0 else 0

    # Calculate coverage percentage
    reference_length = alignment.length
    coverage_percentage = (covered_length / reference_length) * 100 if reference_length != 0 else 0

    return overall_percent_identity, coverage_percentage, covered_length

def merge_intervals(intervals):
    # Sort intervals by start position
    sorted_intervals = sorted(intervals, key=lambda x: x[0])
    merged_intervals = []

    for interval in sorted_intervals:
        if not merged_intervals or merged_intervals[-1][1] < interval[0]:
            merged_intervals.append(interval)
        else:
            merged_intervals[-1] = (merged_intervals[-1][0], max(merged_intervals[-1][1], interval[1]))

    return merged_intervals

def add_to_nested_dict(nested_dict, outer_key, inner_key, value):
    if outer_key not in nested_dict:
        nested_dict[outer_key] = {}  # Initialize a dictionary for this outer key
    if inner_key not in nested_dict[outer_key]:
        nested_dict[outer_key][inner_key] = value  # instantiate inner_key = value


In [224]:
# dir(blast_record)
#['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', 
#'__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'alignments', 
#'application', 'blast_cutoff', 'database', 'database_length', 'database_letters', 'database_name', 'database_sequences', 'date', 'descriptions', 'dropoff_1st_pass', 'effective_database_length', 
#'effective_hsp_length', 'effective_query_length', 'effective_search_space', 'effective_search_space_used', 'expect', 'filter', 'frameshift', 'gap_penalties', 'gap_trigger', 'gap_x_dropoff', 'gap_x_dropoff_final',
#'gapped', 'hsps_gapped', 'hsps_no_gap', 'hsps_prelim_gapped', 'hsps_prelim_gapped_attemped', 'ka_params', 'ka_params_gap', 'matrix', 'multiple_alignment', 'num_good_extends', 'num_hits', 'num_letters_in_database', 
#'num_seqs_better_e', 'num_sequences', 'num_sequences_in_database', 'posted_date', 'query', 'query_id', 'query_length', 'query_letters', 'reference', 'sc_match', 'sc_mismatch', 'threshold', 'version', 'window_size']
####
# dir(alignment)
#['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', 
#'__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'accession', 'hit_def', 'hit_id', 'hsps', 'length', 'title']
test_dict = defaultdict()
for result in wp_results:
    print(result)
    name = Path(result).stem.split("_")[0]
    method = 'longread' if name.endswith(('H','P')) else 'shortread'
    name = name[:-1] if name.endswith(('H','P')) else name
        
    add_to_nested_dict(test_dict, name, method, defaultdict(dict))
    
    with open(result, "r") as blast_file:
       # Parse BLAST XML output
        blast_records = NCBIXML.parse(blast_file)
        for blast_record in blast_records:
            contig = blast_record.query
            query_length = blast_record.query_length
            test_dict[name][method][contig] = {'wp' : [], 'pf32' : []}
            if query_length >= 2500:
                for alignment in blast_record.alignments:
                    alignment_id = alignment.hit_id.split('|')[1]
                    ref_length = alignment.length
                    plasmid_name = parsing_table.get(alignment_id, {}).get('name', 'uh-oh!')
                    overall_percent_identity, coverage_percentage, covered_positions = calculate_percent_identity_and_coverage(alignment)
                    test_dict[name][method][contig]['wp'].append({
                        "alignment_id": alignment_id,
                        "plasmid_name": plasmid_name,
                        "query_length": query_length,
                        "ref_length": ref_length,
                        "overall_percent_identity": overall_percent_identity,
                        "coverage_percentage": coverage_percentage,
                        'covered_positions': covered_positions
                    })

for result in pf32_results:
    print(result)
    name = Path(result).stem.split("_")[0]
    method = 'longread' if name.endswith(('H','P')) else 'shortread'
    name = name[:-1] if name.endswith(('H','P')) else name
    with open(result, "r") as blast_file:
       # Parse BLAST XML output
        blast_records = NCBIXML.parse(blast_file)
        for blast_record in blast_records:
            contig = blast_record.query
            query_length = blast_record.query_length
            if query_length >= 2500:
                for alignment in blast_record.alignments:
                    alignment_id = alignment.hit_id
                    plasmid_name = alignment.hit_id.split('_')[-1]
                    ref_length = alignment.length
                    overall_percent_identity, coverage_percentage, covered_positions = calculate_percent_identity_and_coverage(alignment)
                    test_dict[name][method][contig]['pf32'].append({
                        "alignment_id": alignment_id,
                        "plasmid_name": plasmid_name,
                        "query_length": query_length,
                        "ref_length": ref_length,
                        "overall_percent_identity": overall_percent_identity,
                        "coverage_percentage": coverage_percentage,
                        'covered_positions': covered_positions
                    })

/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/whole_plasmid/URI36_whole_plasmid.xml
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/whole_plasmid/URI91H_whole_plasmid.xml
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/whole_plasmid/URI88H_whole_plasmid.xml
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/whole_plasmid/URI46_whole_plasmid.xml
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/whole_plasmid/UCT31H_whole_plasmid.xml
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/whole_plasmid/UCT32_whole_plasmid.xml
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/whole_plasmid/URI89_whole_plasmid.xml
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/whole_plasmid/UCT109H_whole_plasmid.xml
/home/mf019/borrelia_plasmid_classifier_v3/output/blast_results_v4/whole_plasmid/UCT109_whole_plasmid.xml
/home/mf019/borrelia_plasmid_classifier_v3/output

In [236]:
import csv
# Function to determine the best match
def get_best_match(matches, key):
    best_match = None
    best_score = -1
    for match in matches:
        if match[key] > best_score:
            best_score = match[key]
            best_match = match
    return best_match

with open('best_matches.tsv', 'w', newline='') as tsvfile:
    fieldnames = [
        'name', 'method', 'contig', 
        'wp_alignment_id', 'pf_alignment_id',
        'wp_query_length', 'ref_length',
        'wp_percent_identity', 'wp_coverage_percentage', 
        'wp_plasmid_name', 'pf_plasmid_name',
        'best_hit', 'completeness'
    ]
    writer = csv.DictWriter(tsvfile, fieldnames=fieldnames, delimiter='\t')
    writer.writeheader()

    for name, methods in test_dict.items():
        for method, contigs in methods.items():
            for contig, homologies in contigs.items():
                wp_best_identity = get_best_match(homologies['wp'], 'overall_percent_identity')
                wp_best_coverage = get_best_match(homologies['wp'], 'coverage_percentage')

                pf32_best_identity = None
                if homologies['pf32'] and (not homologies['wp'] or homologies['wp'][0]['query_length'] <= 100000):
                    pf32_best_identity = get_best_match(homologies['pf32'], 'overall_percent_identity')
                
                best_hit = None
                wp_best = None
                pf_best = None
                wp_alignment_id = ''
                pf_alignment_id = ''
                wp_query_length = ''
                ref_length = ''
                wp_percent_identity = ''
                wp_coverage_percentage = ''
                wp_plasmid_name = ''
                pf_plasmid_name = ''

                if wp_best_identity and wp_best_coverage and wp_best_coverage['coverage_percentage'] > wp_best_identity['coverage_percentage']:
                    wp_best = wp_best_coverage
                else:
                    wp_best = wp_best_identity

                if wp_best:
                    wp_alignment_id = wp_best['alignment_id']
                    wp_query_length = wp_best['query_length']
                    ref_length = wp_best['ref_length']
                    wp_percent_identity = wp_best['overall_percent_identity']
                    wp_coverage_percentage = wp_best['coverage_percentage']
                    wp_plasmid_name = wp_best['plasmid_name']

                if pf32_best_identity:
                    pf_alignment_id = pf32_best_identity['alignment_id']
                    pf_plasmid_name = pf32_best_identity['plasmid_name']

                if wp_best and pf32_best_identity:
                    if pf32_best_identity['overall_percent_identity'] > wp_best['overall_percent_identity']:
                        best_hit = 'pf32'
                    else:
                        best_hit = 'wp'
                else:
                    best_hit = 'wp' if wp_best else 'pf32'

                completeness = 'presumed' if wp_best and wp_best['coverage_percentage'] >= 97 else 'incomplete'

                writer.writerow({
                    'name': name,
                    'method': method,
                    'contig': contig,
                    'wp_alignment_id': wp_alignment_id,
                    'pf_alignment_id': pf_alignment_id,
                    'wp_query_length': wp_query_length,
                    'ref_length': ref_length,
                    'wp_percent_identity': wp_percent_identity,
                    'wp_coverage_percentage': wp_coverage_percentage,
                    'wp_plasmid_name': wp_plasmid_name,
                    'pf_plasmid_name': pf_plasmid_name,
                    'best_hit': best_hit,
                    'completeness': completeness
                })

In [237]:
#### END OF NEW CODE!! MOVING TO NEW NOTEBOOK!!! ####

In [ ]:
import pandas

In [ ]:
sr_contig_plasmid_df = pandas.DataFrame(columns=['isolate', 'contig','wp_hit', 'pf32_hit'])
for key in sr_plasmids:
    for item in set(sr_plasmids[key]):
        if item[2] == 'wp':
            new_row = {'isolate': item[0], 'contig': item[1], 'wp_hit': item[3], 'contig_len' : item[4]}#, 'replicon_len' : item[5]}
        else:
            new_row = {'isolate': item[0], 'contig': item[1], 'pf32_hit': item[3], 'contig_len' : item[4]}
        new_row = pandas.DataFrame(new_row, index=[0])
        sr_contig_plasmid_df = pandas.concat([sr_contig_plasmid_df, new_row], ignore_index=True)
sr_df = sr_contig_plasmid_df.groupby(['isolate', 'contig'], as_index = False).agg({
    'wp_hit': lambda x: [item for item in x if str(item) != 'nan'][0],
    'pf32_hit': lambda x: "None" if len(x) == 1 else str([item for item in x if str(item) != 'nan']),
    })
sr_df.sort_index(inplace=True)
sr_df['pf32_hit'] = sr_df['pf32_hit'].str.replace('[', '').str.replace(']', '').str.replace("'", '').replace("nan", 'NaN')
sr_df.to_csv(f'{output_dir}/blast_results_v4/shortread_plasmid_hits_scuffed.tsv', sep='\t', index=False)


In [ ]:
sr_df

In [ ]:
#sr_contig_plasmid_df.groupby(['isolate','contig'], as_index=False).aggregate({
#    'wp_hit': lambda x: [val for val in x if str(val) != 'nan'],
#    'pf32_hit': lambda x: [val for val in x if str(val) != 'nan']})
#sr_contig_plasmid_df
##sr_contig_plasmid_df.to_csv(f'{output_dir}/contig_plasmid_hits.csv')

In [ ]:
lr_contig_plasmid_df = pandas.DataFrame(columns=['isolate', 'contig', 'wp_hit', 'pf32_hit'])
for key in lr_plasmids:
    for item in set(lr_plasmids[key]):
        if item[2] == 'wp':
            new_row = {'isolate': item[0], 'contig': item[1], 'wp_hit': item[3]}
        else:
            new_row = {'isolate': item[0], 'contig': item[1], 'pf32_hit': item[3]}
        new_row = pandas.DataFrame(new_row, index=[0])
        lr_contig_plasmid_df = pandas.concat([lr_contig_plasmid_df, new_row], ignore_index=True)
lr_df = lr_contig_plasmid_df.groupby(['isolate', 'contig'], as_index = False).agg({
    'wp_hit': lambda x: [item for item in x if str(item) != 'nan'][0],
    'pf32_hit': lambda x: "None" if len(x) == 1 else str([item for item in x if str(item) != 'nan'])})
lr_df.sort_index(inplace=True)
lr_df['pf32_hit'] = lr_df['pf32_hit'].str.replace('[', '').str.replace(']', '').str.replace("'", '').str.replace("nan", 'None')
lr_df.to_csv(f'{output_dir}/blast_results_v4/longread_plasmid_hits_scuffed.tsv', sep='\t', index=False)

In [ ]:
lr_df['pf32_hit'].fillna('None')
lr_df

In [ ]:
sr_df[((sr_df['pf32_hit'] != '') & (sr_df['pf32_hit'] != sr_df['wp_hit']))].to_csv(f'{output_dir}/shortread_discrepancies.tsv', sep='\t', index=False)

In [ ]:

lr_df[(lr_df['pf32_hit'] != lr_df['wp_hit']) & (lr_df['pf32_hit'] != '')].to_csv(f'{output_dir}/longread_discrepancies.tsv', sep='\t', index=False)
